In [73]:
import $ivy.`org.apache.spark::spark-sql:2.4.3`
import org.apache.spark.sql._

import $ivy.$                                  

import org.apache.spark.sql._

In [74]:
import org.apache.log4j.{Level,Logger}
Logger.getLogger("org").setLevel(Level.ERROR)

import org.apache.log4j.{Level,Logger}


## Create the sparkSession

In [75]:

val spark = {
            NotebookSparkSession.builder()
            .master("local[*]")
            .getOrCreate()
            }

Getting spark JARs
Creating SparkSession


Spark UI

spark: SparkSession = org.apache.spark.sql.SparkSession@5edd8045

## GetData and parse it 

In [76]:
val parsedData = spark.read
                    .option("header","true")
                    .option("inferSchema","true")
                    .option("nullValue","?")
                    .csv("../linkages/")

csv at cmd75.sc:5

1 / 1

csv at cmd75.sc:5

1 / 1

csv at cmd75.sc:5

5 / 5

csv at cmd75.sc:5

5 / 5

parsedData: DataFrame = [id_1: string, id_2: string ... 10 more fields]

In [77]:
parsedData.take(10)

take at cmd76.sc:1

1 / 1

take at cmd76.sc:1

1 / 1

res76: Array[Row] = Array(
  [3148,8326,1,null,1,null,1,1,1,1,1,true],
  [14055,94934,1,null,1,null,1,1,1,1,1,true],
  [33948,34740,1,null,1,null,1,1,1,1,1,true],
  [946,71870,1,null,1,null,1,1,1,1,1,true],
  [64880,71676,1,null,1,null,1,1,1,1,1,true],
  [25739,45991,1,null,1,null,1,1,1,1,1,true],
  [62415,93584,1,null,1,null,1,1,1,1,0,true],
  [27995,31399,1,null,1,null,1,1,1,1,1,true],
  [4909,12238,1,null,1,null,1,1,1,1,1,true],
  [15161,16743,1,null,1,null,1,1,1,1,1,true]
)

In [78]:
parsedData.printSchema

root
 |-- id_1: string (nullable = true)
 |-- id_2: string (nullable = true)
 |-- cmp_fname_c1: string (nullable = true)
 |-- cmp_fname_c2: string (nullable = true)
 |-- cmp_lname_c1: string (nullable = true)
 |-- cmp_lname_c2: string (nullable = true)
 |-- cmp_sex: string (nullable = true)
 |-- cmp_bd: string (nullable = true)
 |-- cmp_bm: string (nullable = true)
 |-- cmp_by: integer (nullable = true)
 |-- cmp_plz: integer (nullable = true)
 |-- is_match: boolean (nullable = true)



In [79]:
parsedData.count()

count at cmd78.sc:1

5 / 5

count at cmd78.sc:1

5 / 5

count at cmd78.sc:1

1 / 1

count at cmd78.sc:1

1 / 1

res78: Long = 5749133L

In [80]:
parsedData.cache()

res79: DataFrame = [id_1: string, id_2: string ... 10 more fields]

# Basis exploration

In [81]:
parsedData.groupBy("is_match").count().show()

show at cmd80.sc:1

5 / 5

show at cmd80.sc:1

5 / 5

show at cmd80.sc:1

...

show at cmd80.sc:1

75 / 75

show at cmd80.sc:1

75 / 75

+--------+-------+
|is_match|  count|
+--------+-------+
|    null|      1|
|    true|  20931|
|   false|5728201|
+--------+-------+



In [82]:
parsedData.where("is_match=null").show()

+----+----+------------+------------+------------+------------+-------+------+------+------+-------+--------+
|id_1|id_2|cmp_fname_c1|cmp_fname_c2|cmp_lname_c1|cmp_lname_c2|cmp_sex|cmp_bd|cmp_bm|cmp_by|cmp_plz|is_match|
+----+----+------------+------------+------------+------------+-------+------+------+------+-------+--------+
+----+----+------------+------------+------------+------------+-------+------+------+------+-------+--------+



In [83]:
import spark.implicits._
import org.apache.spark.sql.functions._

import spark.implicits._

import org.apache.spark.sql.functions._

In [84]:
parsedData.groupBy("is_match").count().orderBy($"count".desc).show()

show at cmd83.sc:1

5 / 5

show at cmd83.sc:1

5 / 5

show at cmd83.sc:1

200 / 200

show at cmd83.sc:1

200 / 200

+--------+-------+
|is_match|  count|
+--------+-------+
|   false|5728201|
|    true|  20931|
|    null|      1|
+--------+-------+



In [85]:
parsedData.agg(avg($"cmp_sex"),stddev($"cmp_sex")).show()

show at cmd84.sc:1

5 / 5

show at cmd84.sc:1

5 / 5

show at cmd84.sc:1

1 / 1

show at cmd84.sc:1

1 / 1

+------------------+--------------------+
|      avg(cmp_sex)|stddev_samp(cmp_sex)|
+------------------+--------------------+
|0.9550012294607436|  0.2073014119031251|
+------------------+--------------------+



## Create a case class that maps to the columns of the Dataframe. 

org.apache.spark.sql.catalyst.encoders.OuterScopes.addOuterScope(this) This line prevents and error related to the scope of the case classs. Inner vs outer see https://stackoverflow.com/questions/40229953/spark-dataset-example-unable-to-generate-an-encoder-issue

In [115]:
org.apache.spark.sql.catalyst.encoders.OuterScopes.addOuterScope(this)

case class MatchData(
id_1: Option[Int],
id_2: Option[Int],
cmp_fname_c1: Option[Double],
cmp_fname_c2: Option[Double],
cmp_lname_c1: Option[Double],
cmp_lname_c2: Option[Double],
cmp_sex: Option[Int],
cmp_bd: Option[Int],
cmp_bm: Option[Int],
cmp_by: Option[Int],
cmp_plz: Option[Int],
is_match: Option[Boolean]
)

defined class MatchData

In [116]:
parsedData.take(10)

take at cmd115.sc:1

1 / 1

take at cmd115.sc:1

1 / 1

res115: Array[Row] = Array(
  [3148,8326,1.0,null,1.0,null,1,1,1,1,1,true],
  [14055,94934,1.0,null,1.0,null,1,1,1,1,1,true],
  [33948,34740,1.0,null,1.0,null,1,1,1,1,1,true],
  [946,71870,1.0,null,1.0,null,1,1,1,1,1,true],
  [64880,71676,1.0,null,1.0,null,1,1,1,1,1,true],
  [25739,45991,1.0,null,1.0,null,1,1,1,1,1,true],
  [62415,93584,1.0,null,1.0,null,1,1,1,1,0,true],
  [27995,31399,1.0,null,1.0,null,1,1,1,1,1,true],
  [4909,12238,1.0,null,1.0,null,1,1,1,1,1,true],
  [15161,16743,1.0,null,1.0,null,1,1,1,1,1,true]
)

In [117]:
parsedData.printSchema

root
 |-- id_1: integer (nullable = true)
 |-- id_2: integer (nullable = true)
 |-- cmp_fname_c1: double (nullable = true)
 |-- cmp_fname_c2: double (nullable = true)
 |-- cmp_lname_c1: double (nullable = true)
 |-- cmp_lname_c2: double (nullable = true)
 |-- cmp_sex: integer (nullable = true)
 |-- cmp_bd: integer (nullable = true)
 |-- cmp_bm: integer (nullable = true)
 |-- cmp_by: integer (nullable = true)
 |-- cmp_plz: integer (nullable = true)
 |-- is_match: boolean (nullable = true)



In [118]:
val caseClassschema = Encoders.product[MatchData].schema

caseClassschema: types.StructType = StructType(
  StructField("id_1", IntegerType, true, {}),
  StructField("id_2", IntegerType, true, {}),
  StructField("cmp_fname_c1", DoubleType, true, {}),
  StructField("cmp_fname_c2", DoubleType, true, {}),
  StructField("cmp_lname_c1", DoubleType, true, {}),
  StructField("cmp_lname_c2", DoubleType, true, {}),
  StructField("cmp_sex", IntegerType, true, {}),
  StructField("cmp_bd", IntegerType, true, {}),
  StructField("cmp_bm", IntegerType, true, {}),
  StructField("cmp_by", IntegerType, true, {}),
  StructField("cmp_plz", IntegerType, true, {}),
  StructField("is_match", BooleanType, true, {})
)

In [119]:
val parsedData = spark.read.schema(caseClassschema)
                    .option("header","true")
                    .option("nullValue","?")
                    .csv("../linkages/")              

parsedData: DataFrame = [id_1: int, id_2: int ... 10 more fields]

### The above addition of .schema(caseClassschema) was needed to force the schema to match the case class schema rather than rely on the inference of the schema

In [120]:
val scData = parsedData.as[MatchData]

scData: Dataset[MatchData] = [id_1: int, id_2: int ... 10 more fields]

In [121]:
scData.show()

show at cmd120.sc:1

1 / 1

show at cmd120.sc:1

1 / 1

+-----+-----+------------+------------+------------+------------+-------+------+------+------+-------+--------+
| id_1| id_2|cmp_fname_c1|cmp_fname_c2|cmp_lname_c1|cmp_lname_c2|cmp_sex|cmp_bd|cmp_bm|cmp_by|cmp_plz|is_match|
+-----+-----+------------+------------+------------+------------+-------+------+------+------+-------+--------+
| 3148| 8326|         1.0|        null|         1.0|        null|      1|     1|     1|     1|      1|    true|
|14055|94934|         1.0|        null|         1.0|        null|      1|     1|     1|     1|      1|    true|
|33948|34740|         1.0|        null|         1.0|        null|      1|     1|     1|     1|      1|    true|
|  946|71870|         1.0|        null|         1.0|        null|      1|     1|     1|     1|      1|    true|
|64880|71676|         1.0|        null|         1.0|        null|      1|     1|     1|     1|      1|    true|
|25739|45991|         1.0|        null|         1.0|        null|      1|     1|     1|     1|      1|  

In [122]:
scData.printSchema

root
 |-- id_1: integer (nullable = true)
 |-- id_2: integer (nullable = true)
 |-- cmp_fname_c1: double (nullable = true)
 |-- cmp_fname_c2: double (nullable = true)
 |-- cmp_lname_c1: double (nullable = true)
 |-- cmp_lname_c2: double (nullable = true)
 |-- cmp_sex: integer (nullable = true)
 |-- cmp_bd: integer (nullable = true)
 |-- cmp_bm: integer (nullable = true)
 |-- cmp_by: integer (nullable = true)
 |-- cmp_plz: integer (nullable = true)
 |-- is_match: boolean (nullable = true)



In [123]:
case class Score(value: Double){
    def +(oi: Option[Int])={
        Score(value+oi.getOrElse(0))
    }
}

def scoreMatchData(md: MatchData): Double = {
    (Score(md.cmp_lname_c1.getOrElse(0.0))+md.cmp_plz+md.cmp_by+md.cmp_bd+md.cmp_bm).value
}

defined class Score
defined function scoreMatchData

In [124]:
val scored = scData.map(x=>(scoreMatchData(x),x.is_match)).toDF("score","is_match")

scored: DataFrame = [score: double, is_match: boolean]

In [125]:
scored.show()

show at cmd124.sc:1

1 / 1

show at cmd124.sc:1

1 / 1

+-----+--------+
|score|is_match|
+-----+--------+
|  5.0|    true|
|  5.0|    true|
|  5.0|    true|
|  5.0|    true|
|  5.0|    true|
|  5.0|    true|
|  4.0|    true|
|  5.0|    true|
|  5.0|    true|
|  5.0|    true|
|  5.0|    true|
|  5.0|    true|
|  5.0|    true|
|  5.0|    true|
|  5.0|    true|
|  5.0|    true|
|  4.0|    true|
|  5.0|    true|
|  5.0|    true|
|  5.0|    true|
+-----+--------+
only showing top 20 rows



In [126]:
def crossTabs(scored: DataFrame, t:Double): DataFrame = {
    scored.
        selectExpr(s"score>=$t as Above","is_match").
        groupBy("above").
        pivot("is_match", Seq("true","false")).
        count()
}

defined function crossTabs

To get the below code to work I had to modify the case class definition to make the id's and the is_match column  nullable i.e. wrap them in Option[]. There appears to be an empty row that is causingthe problem. Further investigation neeeded.

In [127]:
crossTabs(scored,4.0).show()

show at cmd126.sc:1

5 / 5

show at cmd126.sc:1

5 / 5

show at cmd126.sc:1

...

show at cmd126.sc:1

75 / 75

show at cmd126.sc:1

75 / 75

+-----+-----+-------+
|above| true|  false|
+-----+-----+-------+
| true|20871|    637|
|false|   60|5727564|
+-----+-----+-------+

